In [2]:
import numpy as np
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
import pandas as pd
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import onnx
import onnxruntime as ort
from sklearn.metrics import accuracy_score


# Предподготовка данных

In [4]:

data = pd.read_csv('NSTU_Students_data.csv')
data = data.drop('Курс обучения', axis=1)


In [5]:
binary_fields = {
    'Пол': {'Мужской': True, 'Женский': False},
    'Вы поступили в университет, потому что сами этого хотели?': {'Да': True, 'Нет': False},
    'Вам важно получать хорошие отметки?': {'Да': True, 'Нет': False}
}

for field, mapping in binary_fields.items():
    data[field] = data[field].map(mapping)

In [6]:
fields = {
    'Сумма баллов ЕГЭ за 3 предмета': {'120-150': 1, '151-200': 2, '201-230':3, '231-260': 4, '261-300': 5 }
}
for field, mapping in fields.items():
    data[field] = data[field].map(mapping)

In [7]:
olympiad_fields = [
    'Принимали ли вы участие в олимпиадах по профильным предметам? / Математика',
    'Принимали ли вы участие в олимпиадах по профильным предметам? / Информатика и программирование',
    'Принимали ли вы участие в олимпиадах по профильным предметам? / Физика'
]

for field in olympiad_fields:
    data[field] = data[field].apply(lambda x: True if pd.notna(x) else False)

In [8]:
hobbies_fields = [
    'Какие увлечения у вас были, когда вы учились в школе? / Геймдев, VR, AR',
    'Какие увлечения у вас были, когда вы учились в школе? / Спорт',
    'Какие увлечения у вас были, когда вы учились в школе? / Программирование',
    'Какие увлечения у вас были, когда вы учились в школе? / Робототехника',
    'Какие увлечения у вас были, когда вы учились в школе? / 3D-моделирование',
    'Какие увлечения у вас были, когда вы учились в школе? / Решение математических задач',
    'Какие увлечения у вас были, когда вы учились в школе? / Творчество: музыка, рисование, танцы и т.п.',
    'Какие увлечения у вас были, когда вы учились в школе? / Физика',
]

for field in hobbies_fields:
    data[field] = data[field].apply(lambda x: True if pd.notna(x) else False)

In [9]:
opinion_mapping = {
    'Да, это точно про меня!': 3,
    'Затрудняюсь ответить': 2,
    'Меня это мало интересует': 1
}

for column in data.columns:
    if any(data[column].isin(opinion_mapping.keys())):
        data[column] = data[column].map(opinion_mapping)


In [10]:
it_field = 'Если бы вы могли выбрать только 1 сферу в IT, то что бы выбрали?'
data = pd.get_dummies(data, columns=[it_field], prefix='', prefix_sep='')

In [11]:
target = data['Направление подготовки']

In [12]:
data['target'] = target

In [13]:
data = data.drop('Направление подготовки', axis=1)

In [14]:
data.columns = data.columns.str.replace('Какие увлечения у вас были, когда вы учились в школе?', '', regex=False)
data.columns = data.columns.str.replace('Принимали ли вы участие в олимпиадах по профильным предметам?', 'ОЛ', regex=False)
data.columns = data.columns.str.replace('Насколько вы любите заниматься ...?', 'НЗ', regex=False)
data.columns

Index(['Пол', 'Сумма баллов ЕГЭ за 3 предмета', 'ОЛ / Математика',
       'ОЛ / Информатика и программирование', 'ОЛ / Физика',
       ' / Геймдев, VR, AR', ' / Спорт', ' / Программирование',
       ' / Робототехника', ' / 3D-моделирование',
       ' / Решение математических задач',
       ' / Творчество: музыка, рисование, танцы и т.п.', ' / Физика',
       'НЗ / Веб-разработкой', 'НЗ / Системным администрированием',
       'НЗ / Математикой',
       'НЗ / Работой с компьютерным оборудованием и сборкой ПК',
       'НЗ / Биологией', 'НЗ / Аналитикой данных',
       'НЗ / Разработка графических интерфейсов - GUI',
       'НЗ / Написанием серверной части приложений - Back',
       'НЗ / Написанием клиентской части приложений - Front',
       'НЗ / Экономикой и финансами',
       'НЗ / Поиском ошибок и уязвимостей на сайтах, в играх или приложениях',
       'Вы поступили в университет, потому что сами этого хотели?',
       'Вам важно получать хорошие отметки?', 'QA-тестировщик',
       '

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426 entries, 0 to 425
Data columns (total 37 columns):
 #   Column                                                                Non-Null Count  Dtype 
---  ------                                                                --------------  ----- 
 0   Пол                                                                   426 non-null    bool  
 1   Сумма баллов ЕГЭ за 3 предмета                                        426 non-null    int64 
 2   ОЛ / Математика                                                       426 non-null    bool  
 3   ОЛ / Информатика и программирование                                   426 non-null    bool  
 4   ОЛ / Физика                                                           426 non-null    bool  
 5    / Геймдев, VR, AR                                                    426 non-null    bool  
 6    / Спорт                                                              426 non-null    bool  
 7    / Прогр

In [16]:
for col in data.select_dtypes(include='int64').columns:
    if col != 'Сумма баллов ЕГЭ за 3 предмета':
        data[col] = data[col].astype('category')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426 entries, 0 to 425
Data columns (total 37 columns):
 #   Column                                                                Non-Null Count  Dtype   
---  ------                                                                --------------  -----   
 0   Пол                                                                   426 non-null    bool    
 1   Сумма баллов ЕГЭ за 3 предмета                                        426 non-null    int64   
 2   ОЛ / Математика                                                       426 non-null    bool    
 3   ОЛ / Информатика и программирование                                   426 non-null    bool    
 4   ОЛ / Физика                                                           426 non-null    bool    
 5    / Геймдев, VR, AR                                                    426 non-null    bool    
 6    / Спорт                                                              426 non-null    bool

In [25]:
frequency_table = data['Сумма баллов ЕГЭ за 3 предмета'].value_counts().reset_index()
frequency_table.columns = ['Категория', 'Частота']
counts = frequency_table.set_index('Категория')['Частота'].to_dict()

category_params = {
    1: (135, 4),
    2: (175.5, 3.5),
    3: (215.5, 2.5),
    4: (245.5, 2),
    5: (275, 4) 
}

data['Средние баллы'] = data['Сумма баллов ЕГЭ за 3 предмета'].replace({k: v[0] for k, v in category_params.items()})

categorical_samples = data['Средние баллы'].values

def generate_scores_fixed_count(counts):
    generated_scores = []

    for category, count in counts.items():
        mean, std_dev = category_params[category]

        scores = np.random.normal(mean, std_dev, count)

        if category == 1:
            scores = np.clip(scores, 120, 150)
        elif category == 2:
            scores = np.clip(scores, 151, 200)
        elif category == 3:
            scores = np.clip(scores, 201, 230)
        elif category == 4:
            scores = np.clip(scores, 231, 260)
        elif category == 5:
            scores = np.clip(scores, 261, 280)

        generated_scores.extend(scores)

    return np.array(generated_scores)

# Генерация новых баллов
new_generated_scores = generate_scores_fixed_count(counts)

data['Сумма баллов ЕГЭ за 3 предмета'] = new_generated_scores
data = data.drop('Средние баллы', axis=1)

In [27]:
data.to_csv('NSTU_Students_data_transformed.csv')

In [5]:
ready_data = pd.read_csv('NSTU_Students_data_transformed.csv')
X = ready_data.drop('target', axis=1) 
y = ready_data['target'] 

# Модель для предикта

In [31]:
model_all = AdaBoostClassifier(
    estimator=RandomForestClassifier(n_estimators=3, max_depth=2, random_state=42),
    n_estimators=100,
    algorithm='SAMME',
    random_state=42
)

model_all.fit(X, y)
probabilities_cv = model_all.predict_proba(X)

def calculate_accuracy_top_n(probabilities, y_true, n):
    top_n_predictions = np.argsort(probabilities, axis=1)[:, -n:]
    correct_count = 0
    unique_classes = np.unique(y_true)

    for i, true_label in enumerate(y_true):
        predicted_indices = top_n_predictions[i]
        if true_label in unique_classes[predicted_indices]:
            correct_count += 1

    accuracy = correct_count / len(y_true)
    print(f'Процент правильных ответ для {n} рекомендаций: {accuracy:.2f}')


calculate_accuracy_top_n(probabilities_cv, y, n=1)
calculate_accuracy_top_n(probabilities_cv, y, n=2)
calculate_accuracy_top_n(probabilities_cv, y, n=3)

Процент правильных ответ для 1 рекомендаций: 0.50
Процент правильных ответ для 2 рекомендаций: 0.83
Процент правильных ответ для 3 рекомендаций: 0.98


# ONNX MODEL

## Создаем модель в файл

In [33]:
initial_type = [('float_input', FloatTensorType([None, X.shape[1]]))]
onnx_model = convert_sklearn(model_all, initial_types=initial_type)

with open("model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

## Импортируем модель и проверяем

In [6]:
onnx_model = onnx.load("model.onnx")

onnx.checker.check_model(onnx_model)

ort_session = ort.InferenceSession("model.onnx")

In [10]:
X = X.drop('Unnamed: 0', axis=1)

In [11]:
input_data = X.astype(np.float32).values

In [12]:
ort_inputs = {ort_session.get_inputs()[0].name: input_data}
ort_outs = ort_session.run(None, ort_inputs)

predicted_class = ort_outs[0]

In [27]:
ort_inputs['float_input'][1]

array([  0.     , 246.69734,   1.     ,   0.     ,   1.     ,   0.     ,
         1.     ,   0.     ,   0.     ,   0.     ,   1.     ,   1.     ,
         1.     ,   2.     ,   2.     ,   3.     ,   3.     ,   3.     ,
         3.     ,   2.     ,   2.     ,   2.     ,   3.     ,   2.     ,
         1.     ,   1.     ,   0.     ,   0.     ,   0.     ,   1.     ,
         0.     ,   0.     ,   0.     ,   0.     ,   0.     ,   0.     ],
      dtype=float32)

In [16]:
len(ort_outs)

2

In [19]:
ort_outs[0][0]

'Информатика и вычислительная техника'

In [18]:
ort_outs[1][0]

{'Биотехнические системы и технологии': 0.10958024114370346,
 'Информатика и вычислительная техника': 0.1124861016869545,
 'Информационная безопасность': 0.1110367402434349,
 'Информационная безопасность автоматизированных систем': 0.11135341227054596,
 'Информационные системы и технологии': 0.11115632206201553,
 'Приборостроение': 0.11067236214876175,
 'Прикладная информатика': 0.11181452870368958,
 'Программная инженерия': 0.11122677475214005,
 'Управление в технических системах': 0.11067349463701248}

In [14]:
len(predicted_class)

426

In [37]:
accuracy_score(predicted_class, y)

0.4953051643192488